In [ ]:
import logging
from os.path import join as path_join
from time import time

from joblib import Parallel
from torchvision.transforms import Compose, Resize

from config import ROOT_DIR
from dataset import MfccDataset
from transforms import *

seq_len = 224

trans = Compose([ToTensor(),
                 PaddingSame2d(seq_len=seq_len, value=0)])

start = time()

train_set = MfccDataset(
    root=path_join(ROOT_DIR, "相同文本300"), train=True, transform=trans, n_jobs=-1)

val_set = MfccDataset(
    root=path_join(ROOT_DIR, "不同文本100"), transform=trans, n_jobs=-1)

test_set = MfccDataset(
    root=path_join(ROOT_DIR, "相同文本300"), train=False, transform=trans, n_jobs=-1)

print('total time:', time()-start)

print(train_set)
print(val_set)
print(test_set)

**多线程读取大约能节省100s**

In [ ]:
from collections import OrderedDict

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

%matplotlib inline

torch.manual_seed(17377191)


class Flatten(nn.Module):
    """Note that batch_size is the first dimension"""

    def forward(self, x):
        return x.view(x.size(0), -1)  # [batch, seq_len*input_size]


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=13, stride=13),
            #             nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            #             nn.Dropout(0.1),
            Flatten(),
            nn.Linear(136, 6),

            nn.Softmax()
        )

    def forward(self, x):
        x = x.unsqueeze(dim=1)
        x = self.cnn(x)
        return x

# the training loop


def main(model):
    # hyper params
    T = 20
    learning_rate = 5e-5
    batch_size = 32
    epochs = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Now using {device}")

    train_loader = DataLoader(dataset=train_set+val_set, num_workers=4,
                              shuffle=True, batch_size=batch_size)
    val_loader = DataLoader(dataset=val_set, num_workers=4,
                            shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(dataset=test_set, num_workers=4,
                             shuffle=True, batch_size=batch_size)

    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.to(device)

    # training
    for epoch in range(epochs):  # loop over the dataset multiple times
        print(f"epoch {epoch}")
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (x, y) in enumerate(train_loader, start=0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = Variable(x).type(torch.cuda.FloatTensor)
            labels = Variable(y).type(torch.cuda.LongTensor)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            if i % T == 0:    # print every T mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / T))
                running_loss = 0.0

        with torch.no_grad():
            for x, y in test_loader:
                test_x = x.type(torch.cuda.FloatTensor)
                labels = y.type(torch.cuda.LongTensor)
                outputs = model(test_x)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        print('Accuracy : %d %%' % (100 * acc))
#         plt.plot(epoch, acc)
        print()

    print('Finished Training')


if __name__ == "__main__":
    cnn = CNN()
    main(cnn)

In [ ]:
model_path = "./pickles/cnn.pkl"
torch.save(cnn.state_dict(), model_path)
print('Saved at', model_path)

In [ ]:
cnn.state_dict().get('cnn.0.weight').shape

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=13, stride=13),
            #             nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            #             nn.Dropout(0.1),
            Flatten(),
            nn.Linear(136, 6),

            nn.Softmax()
        )

    def forward(self, x):
        x = x.unsqueeze(dim=1)
#         x = self.cnn(x)
        for layer in self.cnn:
            x = layer(x)
            print(x.shape)
        return x


test_cnn = CNN()
test_cnn(torch.Tensor(13, 224).unsqueeze(dim=0))

In [ ]:
train_loader = iter(DataLoader(dataset=train_set+val_set, num_workers=4,
                               shuffle=True, batch_size=1))
next(train_loader)[0].shape

In [ ]:
torch.Tensor(13, 224).unsqueeze(dim=0).shape